# Getting started with PySkinDose
Welcome! This is a Jupyter Notebook based getting-started tutorial for PySkinDose, which contains tools and scripts for calculating estimates of peak skin dose and to create dose maps for fluoroscopic exams from RDSR data

The tutorial will be updated when more functionality is added or modified.

PyPI: [https://pypi.org/project/pyskindose/](https://pypi.org/project/pyskindose/)

Code repository:[https://github.com/rvbCMTS/PySkinDose](https://github.com/rvbCMTS/PySkinDose)

Documentation:[https://pyskindose.readthedocs.io/en/latest/](https://pyskindose.readthedocs.io/en/latest/)


## PART I: Settings 

This tutorial generates several interactive plots directly within the notebook. If you prefer to view them in a separate browser tab, simply uncomment the following two lines and set `settings.plot.notebook_mode` to False when running PySkinDose.


In [ ]:
# import plotly.io as pio
# pio.renderers.default = "browser"

We need to start our session with adding the class PyskindoseSettings for parsing user defined settings:

In [ ]:
from pyskindose import (
    PyskindoseSettings,
    load_settings_example_json,
    print_available_human_phantoms,
    get_path_to_example_rdsr_files,
    print_example_rdsr_files,
)
from pyskindose.main import main

Once completed, we need to initialize all the user defined settings. Lets load a template of pre-loaded settings (located in `settings_example.json`), then we can change each of the individual settings prior to calculating skin dose estimate.

In [ ]:
# Parse the settings to a setting class:
settings_json = load_settings_example_json()
settings = PyskindoseSettings(settings=settings_json)

Now, all the settings are populated in `settings`. We can print all user defined setting by typing `settings.print_parameters()`:

In [ ]:
settings.print_parameters()

As seen in the above output, `settings`  includes the sections __general__, __phantom__, __plot__ and __normalisation__. 

You can access each of the settings in __general__ by simply typing `settings.mode`, `settings.k_tab_val` etc. For the other sections, include the section name, e.g. `settings.phantom.patient_orientation`.

You can read more about each setting by adressing the corresponding docstring with the `__doc__` attribute. E.g: `settings.__doc__` returns detailed descriptions of the setting in __general__, i.e., "mode", "k_tab_val", "estimate_k_tab", and "rdsr_filename". 

Uncomment any of the following lines for more information regarding each of the subsections in `settings`.


In [ ]:
# print(settings.__doc__)  # uncomment this line to read about settings.general
# print(settings.phantom.__doc__) # uncomment this line to read about settings.phantom
# print(settings.phantom.patient_offset.__doc__)  # uncomment this line to read about settings.phantom.patient_offset
# print(settings.phantom.dimension.__doc__) # uncomment this line to read about settings.phantom.dimension
# print(settings.plot.__doc__) # uncomment this line to read about settings.plot
# print(settings.normalization_settings.__doc__) # uncomment this line to read about settings.normalisation_settings

## PART II: Setup of the skin dose calculation geometry

Now, lets have a look on the geometry in which the skin dose is calculated. PySkinDose has a built in mode for plotting the setup (i.e. the phantom and its position upon the patient support table). This is displayed by running the cell below with `settings.mode` = "plot_setup".

The position of the phantom can be modified by changing the translation parameters in `settings.phantom.patient_offset`. 


In [ ]:
settings = PyskindoseSettings(settings=load_settings_example_json())
settings.mode = "plot_setup"
settings.phantom.model = "cylinder"

# Uncomment any of the following lines to translate the position of the phantom:
# settings.phantom.patient_offset.d_lat = -20
# settings.phantom.patient_offset.d_lon = +10
# settings.phantom.patient_offset.d_ver = -10

main(settings=settings)

In the above output, you will see the phantom and its position upon the support table. The X-ray source, beam, and image receptor is also added in standard position (AP1=AP2=0)


Select your phantom by setting `settings.phantom.model` to either "plane", "cylinder", or "human". Cylindrical phantoms are great for general purposes but for skin dose map visualisation based patient data a human phantom is the best option. If you select "human", you need to specify what human phantom to use. Our current default phantom is named "hudfrid" which is an adult male phantom slightly optimized for skin dose estimations. 

Several more advanced phantoms (e.g. for neurointerventions) are in development. If you select any of the mathematical phantoms (plane or cylinder), then you can tweak settings such as length and radius in `settings.phantom.dimension`. You can also change the dimension of the patient support table and pad.

In [ ]:
settings = PyskindoseSettings(settings=load_settings_example_json())
settings.mode = "plot_setup"

# Select an elliptical cylinder with length 150 cm and a 
# radius of[20, 10] cm
# settings.phantom.model = "cylinder"
# settings.phantom.dimension.cylinder_length = 150
# settings.phantom.dimension.cylinder_radii_a = 20
# settings.phantom.dimension.cylinder_radii_b = 10

# Select a planar phantom with length 120 cm and width 40 cm
# settings.phantom.model = "plane"
# settings.phantom.dimension.plane_length = 120
# settings.phantom.dimension.plane_width = 40

# Select a human phantom
settings.phantom.model = "human"
settings.phantom.human_mesh = "hudfrid"

main(settings=settings)

Use `print_available_human_phantoms()` to show currently available human phantoms. To change human phantom, rerun the previous cell after replacing `hudfrid` with any of the options presented in `print_available_human_phantoms()`


In [ ]:
print_available_human_phantoms()

Once you are satisfied with the selection of patient phantom, and the patient position on the support table, we can move on to examine the contents of an RDSR file prior to calculating the skin dose estimate.

## PART III: Load and examine an RDSR file

PySkinDose contains the following example RDSR files: 

In [ ]:
print_example_rdsr_files()

A useful tool during analysis of individual procedures is to visualize the orientation and size of the x-ray beam in every irradiation event one by one. In the example below the `settings.mode` is changed to `plot_procedure`. With this, you can use the slider in the plot to scroll through each irradiation event in the study. 

Please be aware that the mode `plot_procedure` can be quite computationally intensive when the patient phantom is included in the interactive plot. To reduce this load, you can disable the patient phantom by adjusting the `settings.plot.max_events_for_patient_inclusion` to a value smaller than the number of irradiation events in the RDSR file.

Now, lets load the siemens axiom artis example procedure and visualize the procedure with  `plot_procedure`

In [ ]:
settings = PyskindoseSettings(settings=load_settings_example_json())
settings.mode = "plot_procedure"
settings.phantom.model = "cylinder"

rdsr_data_dir = get_path_to_example_rdsr_files()

# You can set the maximum number of irradiation events for including the
# phantom in the plot. Here, we set it to 0 to reduce memory use.
settings.plot.max_events_for_patient_inclusion = 0

# Change this path to use your own RDSR file
# N.B: If your are using a windows OS, you need to set the path as r raw string,
# for example: 
# selected_rdsr_filepath = Path(r'c:\rdsr_files\file_name.dcm')
selected_rdsr_filepath = rdsr_data_dir / "siemens_axiom_example_procedure.dcm"

main(settings=settings, file_path=selected_rdsr_filepath)

To use your own RDSR file, just change the `file_path` to the location of your RDSR file.

## PART IV: Calculate and plot a dosemap

At this point, you should be satisfied with your choice of patient phantom, the phantom position on the support table and also understand the contents in the selected RDSR-file (from the `settings.plot.plot_dosemap` function).

Now we can finally address the purpose of PySkinDose, which is to calculate the skin dose estimate and visualize the result in a dosemap plot. 

A plot will illustrate the skin dose estimation map based on information in the RDSR-file together with available correction factors for the specific X-ray device. The plot is interactive so the phantom can rotate and if you hover the mouse over the phantom an estimate of skin dose will appear.

In [ ]:
settings = PyskindoseSettings(settings=load_settings_example_json())
settings.mode = "calculate_dose"
settings.output_format = 'html' # set output format to html
settings.plot.plot_dosemap = True # enable dosemap plot
settings.phantom.model = "human"
settings.phantom.human_mesh = "hudfrid"

settings.phantom.patient_offset.d_lat = -35
 
main(settings=settings, file_path=rdsr_data_dir / "siemens_axiom_example_procedure.dcm")


If you want to examine, analyze or save the output of the calculated skin dose estimation, you need to change output mode to either __dict__ or __json__:

In [ ]:
settings = PyskindoseSettings(settings=load_settings_example_json())
settings.mode = "calculate_dose"
settings.output_format = 'dict'

rdsr_data_dir = get_path_to_example_rdsr_files()
output = main(settings=settings, file_path=rdsr_data_dir / "siemens_axiom_example_procedure.dcm")

print(f'estimated psd {output["psd"].round(1)} mGy')

The output (in this case a dictionary) contains the following parameters from the
conducted skin dose calculation.
- **table**
    - parameters regarding the patient support table
- **pad**
    - parameters regarding the patient support pad
- **patient**
    - parameters regarding the phantom
- **dose_map**
    - skin patch index and estimated skin dose value for each patch on the phantom
- **psd**
    - Estimated peak skin dose, i.e., max(dose_map)
- **air_kerma**
    - Air kerma, i.e., psd but without the correction factors
- **events**
    - parameters on translation, rotation for each irradiation event
- **corrections**
    - The correction factor used to convert air kerma to skin dose. This includes
        - backscatter correction
        - medium correction
        - table correction
        - inverse square law correction


## Adding/Altering correction factors

The fine-tuning and optimization of correction factors are, and will always be, a work in progress. Currently, our priority is to implement compliance across additional X-ray devices. One key correction that can be easily adjusted is the estimated attenuation for the patient support table and pad. This can be configured by setting `k_tab_val` to a value between 0 and 1, with 1 indicating that the X-ray beam is not attenuated at all by the table and pad.

Lets try to lower the table and pad correction factor (that is, X-ray beam transmission) and observe the difference in the skin dose map. 

In [ ]:
settings = PyskindoseSettings(settings=load_settings_example_json())
settings.mode = "calculate_dose"
settings.output_format = 'html' # set output format to html
settings.plot.plot_dosemap = True # enable dosemap plot
settings.phantom.model = "human"
settings.phantom.human_mesh = "hudfrid"

settings.phantom.patient_offset.d_lat = -35
settings.k_tab_val = 0.5
main(settings=settings, file_path=rdsr_data_dir / "siemens_axiom_example_procedure.dcm")


In the above output, we note that the skin dose estimates on the back of the phantom is decreased due to increase attenuation in the patient support table and pad. However, the skin dose estimates on the side of the phantom remain the same as before since the X-rays never pass the table and pad.


## Further development and contributions

Contributions and collaborations for further development of PySkinDose is greatly appreciated. Please visit the link below for suggested topics.
[https://pyskindose.readthedocs.io/en/latest/user/contribute/](https://pyskindose.readthedocs.io/en/latest/user/contribute.html)

If you encounter issues using PySkinDose please describe them as detailed as possible at the project GIT linked below
[https://github.com/rvbCMTS/PySkinDose/issues/](https://github.com/rvbCMTS/PySkinDose/issues/)